# Streaming and Real-Time Responses

**Building Interactive AI Agents with Live Streaming Capabilities**

---

Welcome to this comprehensive tutorial on **streaming responses** in the Strands framework! This notebook demonstrates how to build AI agents that provide real-time, streaming responses for enhanced user experiences. By the end of this 10-minute tutorial, you'll master the art of creating responsive, interactive AI applications.

### 🎯 What You'll Learn

In this tutorial, you will:
- Understand streaming vs. traditional responses
- Implement real-time streaming agents
- Build interactive chat interfaces
- Handle streaming events and errors
- Optimize for user experience
- Create progress-aware applications

### ⚡ Why Streaming Matters

Streaming responses provide:
- **Instant feedback** - Users see progress immediately
- **Better UX** - No waiting for complete responses
- **Lower latency** - First tokens arrive quickly
- **Interruptibility** - Users can stop generation
- **Progress tracking** - Monitor response generation

## 📦 Step 1: Installing Required Packages

### Overview
Let's install the necessary packages for streaming functionality.

### 📚 Packages We'll Install
- **strands-agents**: Core framework with streaming support
- **asyncio**: For asynchronous streaming
- **IPython.display**: For real-time notebook updates

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools strands-agents-builder -q

print("✅ All packages installed successfully!")
print("   Ready to build streaming agents! ⚡")

## 🔐 Step 2: Setting Up AWS Authentication

### Overview
We'll configure AWS Bedrock with streaming support enabled.

### 🔑 Authentication Options
1. **AWS Profile** (Recommended for development)
2. **Environment Variables**
3. **Direct Credentials** (Less secure)
4. **IAM Roles** (Recommended for production)

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel
import asyncio
from IPython.display import display, HTML, clear_output
import time

# Configure AWS session
session = boto3.Session(
    # aws_access_key_id='your_access_key',
    # aws_secret_access_key='your_secret_key',
    # aws_session_token='your_session_token',  # If using temporary credentials
    # region_name='us-west-2',
    profile_name='default'  # Optional: Use a specific AWS profile
)

# Create a Bedrock model instance with streaming support
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("✅ AWS Bedrock configured successfully!")
print(f"   Model: Claude 3.7 Sonnet")
print(f"   Profile: {session.profile_name}")
print("   Streaming: Enabled ⚡")

## 🌊 Step 3: Understanding Streaming vs. Traditional Responses

### The Difference
Let's compare traditional and streaming responses to understand the benefits.

In [ ]:
# Traditional response (all at once)
traditional_agent = Agent(
    model=bedrock_model,
    system_prompt="You are a helpful assistant. Be concise."
)

print("🔄 Traditional Response (wait for complete response):")
print("=" * 60)

start_time = time.time()
response = traditional_agent("Explain the benefits of streaming in 3 points")
end_time = time.time()

print(response)
print(f"\n⏱️  Total time: {end_time - start_time:.2f} seconds")
print("   ❌ User waited for entire response")

## ⚡ Step 4: Creating a Streaming Agent

### Real-Time Responses
Now let's create an agent that streams responses token by token.

In [ ]:
# Create a streaming agent
streaming_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a helpful streaming assistant.
    Provide clear, structured responses that work well with streaming.
    Use bullet points and clear sections when appropriate."""
)

print("⚡ Streaming Agent created!")
print("   Type: Real-time streaming")
print("   Benefits: Instant feedback, better UX")

# Function to handle streaming
async def stream_response(agent, prompt):
    """Stream response from agent token by token"""
    print("\n🌊 Streaming Response:")
    print("=" * 60)
    
    collected_response = ""
    start_time = time.time()
    first_token_time = None
    
    # Stream the response
    async for chunk in agent.astream(prompt):
        if first_token_time is None:
            first_token_time = time.time()
        
        # In a real application, you'd update the UI here
        collected_response += chunk
        print(chunk, end='', flush=True)
    
    end_time = time.time()
    
    print(f"\n\n⏱️  Time to first token: {first_token_time - start_time:.2f} seconds")
    print(f"   Total time: {end_time - start_time:.2f} seconds")
    print("   ✅ User saw progress immediately!")
    
    return collected_response

# Test streaming
await stream_response(streaming_agent, "Explain the benefits of streaming in 3 points")

## 💬 Step 5: Building an Interactive Chat Interface

### Live Chat Experience
Let's create a chat interface that updates in real-time as the agent responds.

In [ ]:
class StreamingChatInterface:
    """Interactive chat interface with streaming responses"""
    
    def __init__(self, agent):
        self.agent = agent
        self.conversation_history = []
    
    async def chat(self, user_input):
        """Handle a chat interaction with streaming"""
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": user_input})
        
        # Display user message
        display(HTML(f"<div style='background-color:#e3f2fd; padding:10px; margin:5px; border-radius:5px;'>"
                    f"<strong>You:</strong> {user_input}</div>"))
        
        # Create placeholder for assistant response
        response_html = "<div style='background-color:#f5f5f5; padding:10px; margin:5px; border-radius:5px;'>"
        response_html += "<strong>Assistant:</strong> <span id='response'></span>"
        response_html += "<span id='cursor' style='animation: blink 1s infinite;'>▊</span></div>"
        response_html += "<style>@keyframes blink { 0% {opacity: 1;} 50% {opacity: 0;} 100% {opacity: 1;} }</style>"
        
        display(HTML(response_html))
        
        # Stream the response
        collected_response = ""
        async for chunk in self.agent.astream(user_input):
            collected_response += chunk
            # In a real web app, you'd update the DOM here
            # For notebook, we'll just print
            print(chunk, end='', flush=True)
        
        # Add assistant response to history
        self.conversation_history.append({"role": "assistant", "content": collected_response})
        
        return collected_response

# Create chat interface
chat_interface = StreamingChatInterface(streaming_agent)

print("💬 Interactive Chat Interface Ready!")
print("   Features: Real-time updates, typing indicator, conversation history")

# Test the chat interface
await chat_interface.chat("What are the key advantages of streaming AI responses?")

## 📊 Step 6: Progress Tracking and Metrics

### Monitoring Stream Performance
Let's implement progress tracking to monitor streaming performance.

In [ ]:
class StreamingMetrics:
    """Track streaming performance metrics"""
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.start_time = None
        self.first_token_time = None
        self.end_time = None
        self.token_count = 0
        self.chunk_times = []
    
    def start(self):
        self.start_time = time.time()
    
    def record_chunk(self):
        current_time = time.time()
        if self.first_token_time is None:
            self.first_token_time = current_time
        self.chunk_times.append(current_time)
        self.token_count += 1
    
    def end(self):
        self.end_time = time.time()
    
    def get_metrics(self):
        if not self.start_time or not self.end_time:
            return None
        
        total_time = self.end_time - self.start_time
        time_to_first_token = self.first_token_time - self.start_time if self.first_token_time else 0
        
        # Calculate inter-token delays
        inter_token_delays = []
        for i in range(1, len(self.chunk_times)):
            delay = self.chunk_times[i] - self.chunk_times[i-1]
            inter_token_delays.append(delay)
        
        avg_delay = sum(inter_token_delays) / len(inter_token_delays) if inter_token_delays else 0
        
        return {
            "total_time": total_time,
            "time_to_first_token": time_to_first_token,
            "token_count": self.token_count,
            "tokens_per_second": self.token_count / total_time if total_time > 0 else 0,
            "avg_inter_token_delay": avg_delay
        }

# Test with metrics
async def stream_with_metrics(agent, prompt):
    """Stream response with detailed metrics"""
    metrics = StreamingMetrics()
    
    print(f"📊 Streaming with Metrics: '{prompt}'")
    print("=" * 80)
    
    metrics.start()
    response = ""
    
    async for chunk in agent.astream(prompt):
        metrics.record_chunk()
        response += chunk
        print(chunk, end='', flush=True)
    
    metrics.end()
    
    # Display metrics
    results = metrics.get_metrics()
    print("\n\n📈 Streaming Metrics:")
    print(f"   ⏱️  Time to first token: {results['time_to_first_token']:.3f}s")
    print(f"   ⏱️  Total time: {results['total_time']:.2f}s")
    print(f"   📝 Tokens streamed: {results['token_count']}")
    print(f"   ⚡ Tokens per second: {results['tokens_per_second']:.1f}")
    print(f"   📊 Avg inter-token delay: {results['avg_inter_token_delay']:.3f}s")
    
    return response, results

# Run test with metrics
response, metrics = await stream_with_metrics(
    streaming_agent, 
    "Write a short story about a robot learning to paint (3 paragraphs)"
)

## 🛠️ Step 7: Handling Streaming with Tools

### Streaming Tool-Equipped Agents
Let's explore how streaming works with tool-calling agents.

In [ ]:
from strands import tool
import random

# Define tools
@tool
def generate_plot_point():
    """Generate a random plot point for a story"""
    plot_points = [
        "discovers a hidden treasure map",
        "meets a mysterious stranger",
        "finds an ancient artifact",
        "receives an urgent message",
        "witnesses something impossible"
    ]
    return random.choice(plot_points)

@tool
def generate_character_name():
    """Generate a random character name"""
    first_names = ["Alex", "Jordan", "Sam", "Casey", "Morgan"]
    last_names = ["Stone", "Rivers", "Sky", "Winter", "Fox"]
    return f"{random.choice(first_names)} {random.choice(last_names)}"

# Create streaming agent with tools
streaming_tool_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a creative writing assistant with tools.
    Use the available tools to enhance your stories with dynamic elements.
    Stream your responses for a better reading experience.""",
    tools=[generate_plot_point, generate_character_name]
)

print("🛠️ Streaming Tool Agent created!")
print("   Tools: generate_plot_point, generate_character_name")
print("   Capability: Real-time story generation with dynamic elements")

# Test streaming with tools
print("\n📖 Generating Interactive Story...")
print("=" * 80)

async for chunk in streaming_tool_agent.astream(
    "Write a short adventure story. Use tools to generate the main character's name and a key plot point."
):
    print(chunk, end='', flush=True)
    # Small delay to simulate realistic streaming
    await asyncio.sleep(0.01)

## ⚠️ Step 8: Error Handling in Streaming

### Graceful Error Management
Streaming requires special error handling considerations.

In [ ]:
class SafeStreamingAgent:
    """Streaming agent with comprehensive error handling"""
    
    def __init__(self, agent):
        self.agent = agent
    
    async def safe_stream(self, prompt, max_retries=3):
        """Stream with error handling and retries"""
        for attempt in range(max_retries):
            try:
                print(f"\n🔄 Streaming (Attempt {attempt + 1}/{max_retries})...")
                
                response = ""
                timeout_counter = 0
                
                async for chunk in self.agent.astream(prompt):
                    # Simulate potential timeout check
                    timeout_counter += 1
                    if timeout_counter > 1000:  # Safety limit
                        raise TimeoutError("Stream timeout")
                    
                    response += chunk
                    print(chunk, end='', flush=True)
                
                print("\n✅ Stream completed successfully!")
                return response
                
            except TimeoutError as e:
                print(f"\n⏱️ Timeout error: {e}")
                if attempt < max_retries - 1:
                    print("   Retrying...")
                    await asyncio.sleep(1)
                else:
                    print("   ❌ Max retries reached")
                    return None
                    
            except Exception as e:
                print(f"\n❌ Streaming error: {e}")
                if attempt < max_retries - 1:
                    print("   Retrying...")
                    await asyncio.sleep(1)
                else:
                    print("   ❌ Max retries reached")
                    return None
    
    async def stream_with_fallback(self, prompt):
        """Stream with fallback to non-streaming"""
        try:
            # Try streaming first
            return await self.safe_stream(prompt)
        except:
            print("\n⚠️ Streaming failed, falling back to traditional response...")
            # Fallback to non-streaming
            return self.agent(prompt)

# Create safe streaming wrapper
safe_agent = SafeStreamingAgent(streaming_agent)

# Test error handling
response = await safe_agent.safe_stream("Explain error handling in streaming systems")

## 🎯 Step 9: Building a Real-Time Dashboard

### Live Monitoring Interface
Let's create a dashboard that shows streaming progress in real-time.

In [ ]:
class StreamingDashboard:
    """Real-time dashboard for streaming responses"""
    
    def __init__(self):
        self.sessions = []
    
    async def monitor_stream(self, agent, prompt, session_name):
        """Monitor a streaming session with live updates"""
        session = {
            "name": session_name,
            "prompt": prompt,
            "start_time": time.time(),
            "chunks": [],
            "status": "streaming"
        }
        
        print(f"\n📊 STREAMING DASHBOARD - {session_name}")
        print("=" * 80)
        print(f"📝 Prompt: {prompt}")
        print("\n🔄 Status: STREAMING")
        print("\n📜 Response:")
        print("-" * 40)
        
        response = ""
        chunk_count = 0
        
        try:
            async for chunk in agent.astream(prompt):
                chunk_count += 1
                response += chunk
                session["chunks"].append({
                    "text": chunk,
                    "time": time.time() - session["start_time"]
                })
                
                # Display chunk
                print(chunk, end='', flush=True)
                
                # Update progress bar (simulated)
                if chunk_count % 10 == 0:
                    elapsed = time.time() - session["start_time"]
                    print(f"\n[{elapsed:.1f}s - {chunk_count} chunks]", end='')
            
            session["status"] = "completed"
            session["end_time"] = time.time()
            session["response"] = response
            
        except Exception as e:
            session["status"] = f"error: {e}"
            session["end_time"] = time.time()
        
        # Display summary
        print("\n" + "-" * 40)
        print(f"\n✅ Status: {session['status'].upper()}")
        print(f"⏱️  Duration: {session['end_time'] - session['start_time']:.2f}s")
        print(f"📦 Chunks received: {len(session['chunks'])}")
        print(f"📏 Response length: {len(response)} characters")
        
        self.sessions.append(session)
        return session
    
    def get_analytics(self):
        """Get streaming analytics across all sessions"""
        if not self.sessions:
            return "No sessions recorded"
        
        total_sessions = len(self.sessions)
        successful = sum(1 for s in self.sessions if s["status"] == "completed")
        avg_duration = sum(s.get("end_time", 0) - s["start_time"] for s in self.sessions) / total_sessions
        
        print("\n📈 STREAMING ANALYTICS")
        print("=" * 50)
        print(f"📊 Total sessions: {total_sessions}")
        print(f"✅ Successful: {successful} ({successful/total_sessions*100:.1f}%)")
        print(f"⏱️  Avg duration: {avg_duration:.2f}s")

# Create dashboard
dashboard = StreamingDashboard()

# Test dashboard with a session
session = await dashboard.monitor_stream(
    streaming_agent,
    "Explain the benefits of real-time streaming dashboards",
    "Dashboard Demo"
)

# Show analytics
dashboard.get_analytics()

## 🚀 Step 10: Production Best Practices

### Building Production-Ready Streaming
Let's explore best practices for deploying streaming agents in production.

In [ ]:
print("🚀 PRODUCTION BEST PRACTICES FOR STREAMING")
print("=" * 60)

# Best practices guide
best_practices = {
    "⚡ Performance Optimization": [
        "Implement connection pooling for API calls",
        "Use WebSockets for real-time communication",
        "Enable HTTP/2 for multiplexing",
        "Implement client-side buffering",
        "Consider CDN for global distribution"
    ],
    "🛡️ Error Handling & Resilience": [
        "Implement exponential backoff for retries",
        "Add circuit breakers for failing services",
        "Provide graceful degradation options",
        "Log errors comprehensively",
        "Monitor stream health metrics"
    ],
    "🔐 Security Considerations": [
        "Implement rate limiting per user",
        "Add authentication for streaming endpoints",
        "Validate and sanitize all inputs",
        "Use HTTPS for all connections",
        "Implement timeout limits"
    ],
    "📊 Monitoring & Analytics": [
        "Track time to first byte (TTFB)",
        "Monitor streaming completion rates",
        "Log token generation speed",
        "Track user engagement metrics",
        "Set up alerting for anomalies"
    ],
    "🎯 User Experience": [
        "Show typing indicators during streaming",
        "Implement smooth text rendering",
        "Add progress indicators for long responses",
        "Enable stream interruption/cancellation",
        "Provide fallback for poor connections"
    ]
}

for category, practices in best_practices.items():
    print(f"\n{category}")
    for practice in practices:
        print(f"   • {practice}")

# Example production configuration
print("\n\n📋 Example Production Configuration")
print("=" * 60)

production_config = {
    "streaming": {
        "enabled": True,
        "timeout_seconds": 30,
        "max_retries": 3,
        "retry_delay_seconds": 1,
        "chunk_size": "token",
        "buffer_size": 1024
    },
    "rate_limiting": {
        "requests_per_minute": 60,
        "concurrent_streams": 5,
        "max_tokens_per_stream": 4096
    },
    "monitoring": {
        "log_level": "INFO",
        "metrics_enabled": True,
        "health_check_interval": 60
    }
}

import json
print(json.dumps(production_config, indent=2))

## 🎉 Congratulations!

### 🏆 What You've Accomplished
In this tutorial, you've mastered:
- ✅ Understanding streaming vs. traditional responses
- ✅ Implementing real-time streaming agents
- ✅ Building interactive chat interfaces
- ✅ Creating progress tracking systems
- ✅ Handling streaming with tools
- ✅ Implementing error handling and resilience
- ✅ Building real-time dashboards
- ✅ Production best practices

### ⚡ The Power of Streaming

You now have the skills to build:
- **Interactive chat applications** with real-time responses
- **Progress-aware interfaces** that keep users engaged
- **Resilient streaming systems** that handle errors gracefully
- **Production-ready deployments** with monitoring and analytics

### 💡 Key Takeaways

1. **User Experience First**: Streaming dramatically improves perceived performance
2. **Error Handling is Critical**: Always implement fallbacks and retries
3. **Monitor Everything**: Track metrics to optimize performance
4. **Tools + Streaming**: Combine for powerful interactive applications

### 🔮 Advanced Techniques

Consider exploring:
- **WebSocket Integration**: For bidirectional streaming
- **Server-Sent Events (SSE)**: For unidirectional streaming
- **gRPC Streaming**: For high-performance applications
- **Reactive Streams**: For backpressure handling
- **Stream Processing**: For real-time analytics

### 📚 Resources

- [Strands Documentation](https://strandsagents.com/0.1.x/)
- [AWS Bedrock Streaming Guide](https://docs.aws.amazon.com/bedrock/)
- [AsyncIO Documentation](https://docs.python.org/3/library/asyncio.html)
- [Real-time Web Technologies](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events)

### 🌟 Next Steps

You're ready to:
1. Build chat applications with streaming responses
2. Create real-time dashboards and monitoring
3. Implement streaming in production applications
4. Optimize performance for global scale
5. Combine streaming with other advanced features

### 🚀 Final Thoughts

Streaming transforms AI applications from static Q&A systems into dynamic, interactive experiences. With the techniques you've learned, you can create applications that feel instantaneous and engaging.

Remember: Great user experiences are built one stream at a time!

Happy streaming! ⚡🤖✨